# 문서 이미지 분류 - Albumentations 버전

## 대회 정보
- **Task**: 문서 이미지 분류 (건강보험증, 여권 등)
- **Train Data**: ~1,500장 | **Test Data**: ~3,000장
- **Metric**: Macro F1 Score | **Framework**: PyTorch + Albumentations

## Albumentations 장점
- 🚀 **더 빠른 속도** (transforms 대비 10-100배)
- 🎨 **80개 이상 증강 기법**
- 📄 **문서 특화** (GridDistortion, Perspective 등)

---

## 🎯 추천 모델 (실험 순서)

### 1단계: Baseline ⭐⭐⭐⭐⭐
```python
CFG.model_name = 'efficientnet_b0'
CFG.augmentation_level = 'medium'
```
**파라미터**: 5M | **속도**: ~1분/epoch | **용도**: 3가지 증강 레벨 테스트

### 2단계: 성능 향상 ⭐⭐⭐⭐⭐
```python
CFG.model_name = 'efficientnet_b1'  # 추천 1순위
# 또는
CFG.model_name = 'efficientnet_b2'  # 추천 2순위
```
**B1**: 7M, ~1.5분/epoch, B0 대비 +2~3% 향상  
**B2**: 9M, ~2분/epoch, B0 대비 +3~5% 향상

### 3단계: 최신 아키텍처 ⭐⭐⭐⭐⭐
```python
CFG.model_name = 'convnext_tiny'
```
**파라미터**: 28M | **속도**: ~2.5분/epoch | **특징**: 최신(2022), B0 대비 +5~7% 향상

### 4단계: 성능 극대화 ⭐⭐⭐⭐
```python
CFG.model_name = 'efficientnet_b3'  # 1순위
CFG.augmentation_level = 'heavy'    # 큰 모델엔 강한 증강
# 또는
CFG.model_name = 'convnext_small'   # 2순위 (최고 성능)
```
**B3**: 12M, ~2.5분/epoch, B0 대비 +5~8% 향상  
**ConvNeXt-Small**: 50M, ~4분/epoch, B0 대비 +7~10% 향상

### 5단계: Transformer (선택) ⭐⭐
```python
CFG.model_name = 'vit_base_patch16_224'
CFG.augmentation_level = 'heavy'  # 필수!
# 또는
CFG.model_name = 'swin_base_patch4_window7_224'
CFG.augmentation_level = 'heavy'  # 필수!
```
**ViT**: 86M, ~5분/epoch | **Swin**: 88M, ~5분/epoch  
**주의**: 1,500장에선 과적합 위험 높음, Heavy 증강 필수, **비추천**

---

## ⚠️ 비추천 모델
- `resnet50` / `resnet101` - EfficientNet보다 비효율적
- `mobilenetv3_large_100` - 속도 빠르지만 성능 낮음
- `vit` / `swin` - 데이터 부족 시 과적합 (10,000장 이상일 때 추천)

---

## 🚀 실험 시나리오

### 시나리오 1: 빠른 실험 (2시간)
1. B0 + Light → 30분 (F1: 0.75)
2. B0 + Medium → 30분 (F1: 0.78)
3. B0 + Heavy → 30분 (F1: 0.81)
4. B1 + Heavy → 40분 (F1: 0.84)

### 시나리오 2: 균형 실험 (4시간)
B0(3가지 증강) → B1 → B2 → ConvNeXt-Tiny → 최고 모델 재학습

### 시나리오 3: 최고 성능 (하루)
B0 증강 최적화 → B1/B2 → ConvNeXt-Tiny → B3+Heavy → ConvNeXt-Small → 앙상블

---

## 💡 증강 레벨 가이드

| 레벨 | 언제 사용? | 특징 |
|------|-----------|------|
| **Light** | 데이터 깨끗/충분 | 빠름, 원본 유지 |
| **Medium** ⭐ | 대부분 경우 (권장) | 균형, 현실적 변형 |
| **Heavy** | 데이터 부족/과적합 | 최대 일반화 |

**팁**: 작은 모델(B0/B1) → medium, 큰 모델(B3/ConvNeXt/Transformer) → heavy

---

## 🔧 트러블슈팅

### GPU 메모리 부족 (OOM Error)
**해결 방법 (우선순위 순):**
1. `CFG.batch_size = 16` (추천, 이미지 사이즈 유지) ⭐
2. `CFG.batch_size = 8` (더 안전)
3. Mixed Precision 사용: `torch.cuda.amp.autocast()`
4. **최후의 수단**: `CFG.img_size = 256` (권장 안 함, 성능 저하)

**⚠️ 주의**: 문서 이미지는 `img_size`를 256 이하로 줄이지 마세요! 글자가 안 보여 분류가 불가능합니다.

### 학습이 너무 느림
- Early Stopping이 있으므로 자동으로 최적화됨 (~15 epoch)
- 수동 조정: `CFG.epochs = 20`

### 성능이 plateau (정체)
- 모델 크기 키우기보다 **증강/하이퍼파라미터 튜닝** 먼저!
- Learning rate 조정, 증강 레벨 변경 시도

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# 필요한 라이브러리 설치
!pip install timm wandb albumentations -q

In [ ]:
import osimport randomimport numpy as npimport pandas as pdfrom PIL import Imageimport cv2from tqdm import tqdmimport warningswarnings.filterwarnings('ignore')import torchimport torch.nn as nnimport torch.optim as optimfrom torch.utils.data import Dataset, DataLoader# Albumentations 임포트import albumentations as Afrom albumentations.pytorch import ToTensorV2import timmfrom sklearn.model_selection import train_test_split, StratifiedKFoldfrom sklearn.metrics import f1_score, confusion_matriximport wandb# ============================# 환경 자동 감지 및 GPU 설정# ============================def get_device():    """맥/윈도우/리눅스 환경 자동 감지 및 최적 디바이스 선택"""    if torch.cuda.is_available():        device = torch.device('cuda')        device_name = torch.cuda.get_device_name(0)        print(f'✓ CUDA 사용 가능')        print(f'  GPU: {device_name}')    elif torch.backends.mps.is_available():        device = torch.device('mps')        print(f'✓ Apple Silicon MPS 사용 가능')        print(f'  Metal Performance Shaders 가속 활성화')    else:        device = torch.device('cpu')        print(f'⚠️ GPU를 사용할 수 없습니다. CPU를 사용합니다.')        return devicedevice = get_device()print(f'\nUsing device: {device}')print(f'PyTorch version: {torch.__version__}')print(f'Albumentations version: {A.__version__}')

In [ ]:
# 환경 자동 감지 (Google Colab vs Local)
import sys

try:
    import google.colab
    IS_COLAB = True
    print("🌐 Running on Google Colab")
except:
    IS_COLAB = False
    print("💻 Running on Local Machine")

## 2. 시드 고정 (재현성)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
# Google Drive 마운트 (Colab only)
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✓ Google Drive mounted")
else:
    print("✓ Local environment - skipping drive mount")

In [ ]:
# WandB 로그인if Config.use_wandb if 'Config' in dir() else True:    if IS_COLAB:        wandb.login()  # Colab에서는 매번 로그인 필요        print("✓ WandB login required on Colab")    else:        # 로컬은 이미 로그인되어 있다고 가정        print("✓ Using local WandB credentials")WANDB_PROJECT = "document-classification"WANDB_ENTITY = None

In [ ]:
MODEL_IMG_SIZES = {    'efficientnet_b0': 384,    'efficientnet_b1': 416,    'efficientnet_b2': 448,    'efficientnet_b3': 512,    'convnext_tiny': 384,    'convnext_small': 384,}class Config:    # 환경별 경로 설정    if IS_COLAB:        data_dir = '/content/drive/MyDrive/deep_learning_IC/data'        drive_model_dir = '/content/drive/MyDrive/deep_learning_IC/models'        save_to_drive = True    else:        data_dir = './data'  # 로컬 데이터 경로        drive_model_dir = './models'  # 로컬 모델 저장 경로        save_to_drive = True  # 로컬에서도 models 폴더에 저장        train_dir = f'{data_dir}/train'    test_dir = f'{data_dir}/test'    train_csv = f'{data_dir}/train.csv'    test_csv = f'{data_dir}/test.csv'        # 로컬 모델 저장 경로    local_model_path = './best_model.pth'  # 현재 디렉토리에 임시 저장        # 모델 설정    model_name = 'efficientnet_b0'    num_classes = 10    img_size = MODEL_IMG_SIZES.get(model_name, 384)        # 학습 설정    epochs = 30    batch_size = 32    learning_rate = 1e-4    weight_decay = 1e-5        # Early Stopping    early_stopping_patience = 3    early_stopping_min_delta = 0.0001        # 데이터 분할    val_ratio = 0.2        # Albumentations 증강 강도 설정    augmentation_level = 'medium'        # WandB    use_wandb = True    wandb_project = WANDB_PROJECT    wandb_entity = WANDB_ENTITY    experiment_name = None    experiment_prefix = None        # 실험 모드 설정    experiment_mode = True  # True면 CIFAR-10 사용, False면 실제 대회 데이터 사용    sample_ratio = 0.1  # 실험 시 사용할 데이터 비율 (0.1 = 10%)        # 기타    num_workers = 2    seed = 42

In [ ]:
# ============================# CIFAR-10 데이터 다운로드 및 준비# ============================if Config.experiment_mode:    print("\n" + "="*70)    print("🧪 실험 모드: CIFAR-10 데이터셋 사용")    print(f"📊 샘플링 비율: {Config.sample_ratio*100:.0f}%")    print("="*70 + "\n")        import torchvision    import torchvision.transforms as transforms    from collections import defaultdict        # CIFAR-10 다운로드    print("📥 CIFAR-10 데이터셋 다운로드 중...")    cifar10_train = torchvision.datasets.CIFAR10(        root='./data/cifar10',         train=True,         download=True    )    cifar10_test = torchvision.datasets.CIFAR10(        root='./data/cifar10',         train=False,         download=True    )        # CIFAR-10 클래스 이름    cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',                        'dog', 'frog', 'horse', 'ship', 'truck']        print(f"✓ Train 샘플: {len(cifar10_train)}개")    print(f"✓ Test 샘플: {len(cifar10_test)}개")    print(f"✓ 클래스: {cifar10_classes}\n")        # 실험용 샘플링 (클래스별 균등 샘플링)    def sample_cifar10(dataset, sample_ratio, seed=42):        """클래스별로 균등하게 샘플링"""        np.random.seed(seed)                # 클래스별로 인덱스 그룹화        class_indices = defaultdict(list)        for idx, (_, label) in enumerate(dataset):            class_indices[label].append(idx)                # 각 클래스에서 sample_ratio만큼 샘플링        sampled_indices = []        for label, indices in class_indices.items():            n_samples = int(len(indices) * sample_ratio)            sampled = np.random.choice(indices, size=n_samples, replace=False)            sampled_indices.extend(sampled)                return sorted(sampled_indices)        # 샘플링 적용    if Config.sample_ratio < 1.0:        train_indices = sample_cifar10(cifar10_train, Config.sample_ratio, Config.seed)        test_indices = sample_cifar10(cifar10_test, Config.sample_ratio, Config.seed)                print(f"📊 샘플링 결과:")        print(f"  Train: {len(cifar10_train)} → {len(train_indices)} ({Config.sample_ratio*100:.0f}%)")        print(f"  Test: {len(cifar10_test)} → {len(test_indices)} ({Config.sample_ratio*100:.0f}%)")    else:        train_indices = list(range(len(cifar10_train)))        test_indices = list(range(len(cifar10_test)))        # 데이터를 Albumentations 형식으로 변환    train_paths = []    train_labels = []    for idx in train_indices:        img, label = cifar10_train[idx]        train_paths.append(img)  # PIL Image 저장        train_labels.append(label)        test_paths = []    test_labels = []    for idx in test_indices:        img, label = cifar10_test[idx]        test_paths.append(img)        test_labels.append(label)        # 클래스 매핑    class_to_idx = {name: idx for idx, name in enumerate(cifar10_classes)}    idx_to_class = {idx: name for name, idx in class_to_idx.items()}        # Config 업데이트    Config.num_classes = 10    Config.img_size = 32  # CIFAR-10은 32x32        print(f"\n✓ CIFAR-10 실험 데이터 준비 완료!")    print(f"  Train: {len(train_paths)}개")    print(f"  Test: {len(test_paths)}개")    print(f"  Image size: {Config.img_size}x{Config.img_size}")    print(f"  Classes: {Config.num_classes}\n")    else:    print("\n" + "="*70)    print("🏆 실제 대회 모드: 대회 데이터셋 사용")    print("="*70 + "\n")        # 기존 데이터 로드 로직 (train.csv + meta.csv)    # 이 부분은 기존 코드 사용    pass

In [ ]:
# 로컬 환경에서 필요한 디렉토리 생성if not IS_COLAB:    os.makedirs(Config.data_dir, exist_ok=True)    os.makedirs(Config.drive_model_dir, exist_ok=True)    print(f"✓ Local directories ready:")    print(f"  - Data: {Config.data_dir}")    print(f"  - Models: {Config.drive_model_dir}")else:    print(f"✓ Colab environment - using drive paths:")    print(f"  - Data: {Config.data_dir}")    print(f"  - Models: {Config.drive_model_dir}")

## 6. Albumentations 데이터셋 클래스

In [ ]:
class AlbumentationsDataset(Dataset):    """    Albumentations를 사용하는 데이터셋 클래스        실험 모드: PIL Image 객체 직접 처리 (CIFAR-10)    대회 모드: 파일 경로에서 이미지 로드    """    def __init__(self, image_paths, labels=None, transform=None):        self.image_paths = image_paths        self.labels = labels        self.transform = transform        def __len__(self):        return len(self.image_paths)        def __getitem__(self, idx):        img_path_or_image = self.image_paths[idx]                # PIL Image 객체인지 파일 경로인지 확인        if isinstance(img_path_or_image, Image.Image):            # CIFAR-10 등 PIL Image 객체인 경우            image = np.array(img_path_or_image)        else:            # 파일 경로인 경우 (기존 방식)            image = cv2.imread(img_path_or_image)            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)                if self.transform:            # Albumentations는 dict 형태로 반환            augmented = self.transform(image=image)            image = augmented['image']                if self.labels is not None:            label = self.labels[idx]            return image, label        else:            return image

## 7. Albumentations 데이터 증강 설정

### 문서 이미지에 특화된 증강 기법

#### Light (약한 증강)
- 깨끗한 문서 이미지에 적합
- 기본적인 색상 조정과 약간의 회전만 적용

#### Medium (중간 증강) - **권장**
- 대부분의 문서 이미지 분류에 적합
- 현실적인 변형들을 시뮬레이션
- 조명 변화, 그림자, 약간의 왜곡 등

#### Heavy (강한 증강)
- 데이터가 매우 부족하거나 높은 다양성이 필요한 경우
- 강한 왜곡, 노이즈, 컷아웃 등 포함
- 과도한 증강은 오히려 성능 저하를 일으킬 수 있으므로 주의

In [ ]:
def get_train_transforms(img_size=224, level='medium'):    """    문서 이미지 분류에 특화된 Albumentations 학습용 증강        Args:        img_size: 입력 이미지 크기        level: 증강 강도 ('light', 'medium', 'heavy')    """        if level == 'light':        return A.Compose([            # 기본 리사이즈            A.Resize(img_size, img_size),                        # 약한 회전 (문서가 약간 기울어진 경우)            A.Rotate(limit=5, p=0.5),                        # 기본 색상 조정            A.RandomBrightnessContrast(                brightness_limit=0.1,                contrast_limit=0.1,                p=0.5            ),                        # 정규화 및 텐서 변환            A.Normalize(                mean=[0.485, 0.456, 0.406],                std=[0.229, 0.224, 0.225]            ),            ToTensorV2()        ])        elif level == 'medium':        return A.Compose([            # 기본 리사이즈            A.Resize(img_size, img_size),                        # 수평 뒤집기 (일부 문서는 대칭인 경우)            A.HorizontalFlip(p=0.3),                        # 회전 (문서가 기울어진 경우)            A.Rotate(limit=10, border_mode=cv2.BORDER_CONSTANT, value=255, p=0.5),                        # 원근 변환 (문서를 비스듬히 촬영한 경우)            A.Perspective(scale=(0.05, 0.1), p=0.3),                        # 색상 및 밝기 조정 (조명 변화)            A.OneOf([                A.RandomBrightnessContrast(                    brightness_limit=0.2,                    contrast_limit=0.2,                    p=1.0                ),                A.HueSaturationValue(                    hue_shift_limit=10,                    sat_shift_limit=20,                    val_shift_limit=10,                    p=1.0                ),                A.CLAHE(clip_limit=2.0, p=1.0),            ], p=0.5),                        # 그림자 효과 (조명이 불균일한 경우)            A.RandomShadow(p=0.2),                        # 약간의 블러 (초점이 맞지 않은 경우)            A.OneOf([                A.GaussianBlur(blur_limit=(3, 5), p=1.0),                A.MotionBlur(blur_limit=3, p=1.0),            ], p=0.2),                        # 약한 노이즈            A.GaussNoise(var_limit=(10.0, 30.0), p=0.2),                        # 정규화 및 텐서 변환            A.Normalize(                mean=[0.485, 0.456, 0.406],                std=[0.229, 0.224, 0.225]            ),            ToTensorV2()        ])        elif level == 'heavy':        return A.Compose([            # 기본 리사이즈            A.Resize(img_size, img_size),                        # 수평/수직 뒤집기            A.HorizontalFlip(p=0.5),            A.VerticalFlip(p=0.2),                        # 강한 회전            A.Rotate(limit=15, border_mode=cv2.BORDER_CONSTANT, value=255, p=0.7),                        # 강한 원근/왜곡 변환            A.OneOf([                A.Perspective(scale=(0.05, 0.15), p=1.0),                A.GridDistortion(num_steps=5, distort_limit=0.3, p=1.0),                A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0),            ], p=0.5),                        # 강한 색상 변환            A.OneOf([                A.RandomBrightnessContrast(                    brightness_limit=0.3,                    contrast_limit=0.3,                    p=1.0                ),                A.HueSaturationValue(                    hue_shift_limit=20,                    sat_shift_limit=30,                    val_shift_limit=20,                    p=1.0                ),                A.CLAHE(clip_limit=4.0, p=1.0),                A.ColorJitter(p=1.0),            ], p=0.7),                        # 그림자 및 조명 효과            A.RandomShadow(p=0.3),                        # 블러 효과            A.OneOf([                A.GaussianBlur(blur_limit=(3, 7), p=1.0),                A.MotionBlur(blur_limit=5, p=1.0),                A.MedianBlur(blur_limit=5, p=1.0),            ], p=0.3),                        # 노이즈            A.OneOf([                A.GaussNoise(var_limit=(20.0, 50.0), p=1.0),                A.ISONoise(p=1.0),            ], p=0.3),                        # 컷아웃 (일부 영역 제거)            A.CoarseDropout(                max_holes=8,                max_height=int(img_size * 0.1),                max_width=int(img_size * 0.1),                fill_value=255,                p=0.3            ),                        # 정규화 및 텐서 변환            A.Normalize(                mean=[0.485, 0.456, 0.406],                std=[0.229, 0.224, 0.225]            ),            ToTensorV2()        ])        else:        raise ValueError(f"Unknown augmentation level: {level}. Use 'light', 'medium', or 'heavy'.")def get_valid_transforms(img_size=224):    """    검증/테스트용 변환 (증강 없음)    """    return A.Compose([        A.Resize(img_size, img_size),        A.Normalize(            mean=[0.485, 0.456, 0.406],            std=[0.229, 0.224, 0.225]        ),        ToTensorV2()    ])# Transform 생성train_transform = get_train_transforms(    img_size=Config.img_size,    level=Config.augmentation_level)val_transform = get_valid_transforms(img_size=Config.img_size)print(f"Augmentation level: {Config.augmentation_level}")print(f"Train transforms: {len(train_transform)} operations")print(f"Valid transforms: {len(val_transform)} operations")

---

## 📌 Albumentations 증강 레벨 선택 가이드

### 🎯 어떤 레벨을 선택해야 할까요?

**CFG 클래스의 `augmentation_level` 파라미터를 변경하면 됩니다!**

```python
# 섹션 5의 CFG 클래스에서
CFG.augmentation_level = 'medium'  # 이 한 줄만 변경하면 됩니다!
```

---

### 💡 레벨별 특징 및 추천 상황

#### 🟢 **Light** - 약한 증강
```python
CFG.augmentation_level = 'light'
```

**적용 증강:**
- 약한 회전 (±5도)
- 기본 밝기/대비 조정

**추천 상황:**
- ✅ 문서 이미지가 **매우 깨끗하고 품질이 좋은 경우**
- ✅ 데이터가 **충분히 많은 경우** (1000장 이상)
- ✅ 학습 데이터가 **실제 테스트 환경과 매우 유사한 경우**
- ✅ 과도한 증강으로 인해 성능이 저하되는 경우

**장점:** 빠른 학습 속도, 원본 데이터의 특성 유지  
**단점:** 일반화 성능이 낮을 수 있음

---

#### 🟡 **Medium** - 중간 증강 ⭐ **권장!**
```python
CFG.augmentation_level = 'medium'  # 기본값
```

**적용 증강:**
- 수평 뒤집기 (30%)
- 회전 (±10도)
- 원근 변환 (Perspective)
- 밝기/대비/색조 조정
- 그림자 효과
- 약한 블러 및 노이즈

**추천 상황:**
- ✅ **대부분의 문서 이미지 분류 작업** (가장 범용적)
- ✅ 데이터가 **중간 정도인 경우** (500-2000장)
- ✅ 실제 환경에서 **조명이나 각도가 다양한 경우**
- ✅ 스캔/촬영 품질이 **일정하지 않은 경우**
- ✅ **첫 실험으로 권장!** 이후 성능을 보고 조정

**장점:** 성능과 일반화의 균형, 현실적인 변형 시뮬레이션  
**단점:** 없음 (가장 안정적)

---

#### 🔴 **Heavy** - 강한 증강
```python
CFG.augmentation_level = 'heavy'
```

**적용 증강:**
- 수평/수직 뒤집기
- 강한 회전 (±15도)
- 강한 원근/그리드 왜곡 변환
- 강한 색상 변환
- 그림자 및 강한 블러
- 노이즈 추가
- **CoarseDropout** (일부 영역 제거)

**추천 상황:**
- ✅ 데이터가 **매우 부족한 경우** (500장 미만)
- ✅ **과적합(Overfitting)이 심한 경우**
- ✅ 테스트 환경이 **학습 환경과 매우 다른 경우**
- ✅ 문서 품질이 **다양하고 예측 불가능한 경우**
- ⚠️ Medium으로 시도 후 성능이 낮을 때 고려

**장점:** 최대 일반화, 강한 정규화 효과  
**단점:** 과도한 증강으로 성능 저하 가능, 학습 시간 증가

---

### 🔄 실험 워크플로우 추천

#### **단계 1: Medium으로 시작 (Baseline)**
```python
CFG.augmentation_level = 'medium'
```
→ 학습 후 성능 확인

#### **단계 2: 성능에 따라 조정**

**Case A: Train 정확도 >> Val 정확도 (과적합)**
```python
CFG.augmentation_level = 'heavy'  # 더 강한 증강으로 정규화
```

**Case B: Train/Val 모두 높지만 Test가 낮음**
```python
CFG.augmentation_level = 'heavy'  # 더 다양한 변형 학습
```

**Case C: Train/Val 모두 낮음**
```python
CFG.augmentation_level = 'light'  # 증강 약화, 모델/하이퍼파라미터 점검
```

**Case D: 성능이 충분히 좋음**
```python
# Medium 유지 또는 Light로 미세 조정
```

---

### 📊 WandB에서 비교하기

**각 레벨별로 실험을 진행하면 자동으로 다른 이름으로 저장됩니다:**

```python
# Light 실험
CFG.augmentation_level = 'light'
# → efficientnet_b0_alb_light_001

# Medium 실험
CFG.augmentation_level = 'medium'
# → efficientnet_b0_alb_medium_001

# Heavy 실험
CFG.augmentation_level = 'heavy'
# → efficientnet_b0_alb_heavy_001
```

**WandB 대시보드에서 세 실험을 선택하고 비교하세요!**

---

### 🎨 증강 시각화로 확인하기

**섹션 9**의 증강 시각화 코드를 실행하면 각 레벨의 증강 결과를 눈으로 확인할 수 있습니다!

```python
# 데이터 로드 후 실행
visualize_augmentations(train_dataset, idx=0, samples=4)
```

**증강이 너무 강해 보이면** → Light로 변경  
**증강이 너무 약해 보이면** → Heavy로 변경

---

### 💻 빠른 변경 예시

섹션 5의 CFG 클래스로 돌아가서 한 줄만 수정:

```python
class CFG:
    # ... (다른 설정들)
    
    # 이 한 줄만 바꾸면 됩니다!
    augmentation_level = 'medium'  # 'light', 'medium', 'heavy' 중 선택
    
    # ... (다른 설정들)
```

그리고 노트북을 처음부터 다시 실행! 🚀

---

## 8. 데이터 로드 및 전처리

**주의**: 이 코드는 제공된 데이터셋 구조에 맞게 작성되었습니다.

데이터 구조:
```
data/
├── train.csv       (ID, target)
├── meta.csv        (target, class_name)
├── train/          (이미지 파일들)
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
└── test/           (테스트 이미지 파일들)
    ├── test1.jpg
    └── ...
```

In [ ]:
# 데이터 로드 함수 (train.csv + meta.csv 기반)def load_data_from_csv(train_csv_path, meta_csv_path, train_dir):    """    train.csv와 meta.csv를 읽어서 데이터 로드    Args:        train_csv_path: 'train.csv' 경로 (ID, target)        meta_csv_path: 'meta.csv' 경로 (target, class_name)        train_dir: 학습 이미지 폴더 경로    Returns:        image_paths, labels, class_to_idx    """    # train.csv 읽기 (ID, target)    train_df = pd.read_csv(train_csv_path)    print(f"train.csv loaded: {len(train_df)} entries")    # meta.csv 읽기 (target, class_name)    meta_df = pd.read_csv(meta_csv_path)    print(f"meta.csv loaded: {len(meta_df)} classes")    # class_to_idx 매핑 생성 (class_name → target)    class_to_idx = dict(zip(meta_df['class_name'], meta_df['target']))    # idx_to_class 매핑 생성 (target → class_name)    idx_to_class = dict(zip(meta_df['target'], meta_df['class_name']))    # 이미지 경로와 라벨 리스트 생성    image_paths = []    labels = []    missing_count = 0    for _, row in train_df.iterrows():        img_id = row['ID']        target = row['target']        # 이미지 경로 생성 (확장자가 포함되어 있을 수도 있음)        img_path = os.path.join(train_dir, img_id)        # 파일이 실제로 존재하는지 확인        if os.path.exists(img_path):            image_paths.append(img_path)            labels.append(target)        else:            # 확장자를 시도해보기            found = False            for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:                img_path_with_ext = os.path.join(train_dir, img_id + ext) if not img_id.endswith(ext) else img_path                if os.path.exists(img_path_with_ext):                    image_paths.append(img_path_with_ext)                    labels.append(target)                    found = True                    break            if not found:                missing_count += 1    if missing_count > 0:        print(f"Warning: {missing_count} images from train.csv not found in {train_dir}")    print(f"\nSuccessfully loaded {len(image_paths)} images")    print(f"Number of classes: {len(class_to_idx)}")    print(f"\nClasses:")    for class_name, idx in sorted(class_to_idx.items(), key=lambda x: x[1]):        print(f"  [{idx}] {class_name}")    return image_paths, labels, class_to_idx# 학습 데이터 로드train_csv_path = './data/train.csv'meta_csv_path = './data/meta.csv'if os.path.exists(train_csv_path) and os.path.exists(meta_csv_path) and os.path.exists(Config.train_dir):    train_paths, train_labels, class_to_idx = load_data_from_csv(train_csv_path, meta_csv_path, Config.train_dir)    # Config.num_classes 업데이트    Config.num_classes = len(class_to_idx)else:    missing_files = []    if not os.path.exists(train_csv_path):        missing_files.append(train_csv_path)    if not os.path.exists(meta_csv_path):        missing_files.append(meta_csv_path)    if not os.path.exists(Config.train_dir):        missing_files.append(Config.train_dir)    print(f"Error: Missing required files/directories:")    for f in missing_files:        print(f"  - {f}")    print("\nPlease upload your data or modify the paths in Config.")

In [ ]:
import matplotlib.pyplot as plt

def visualize_random_samples(image_paths, labels, class_to_idx, num_samples=5):
    """
    데이터셋에서 랜덤으로 샘플을 추출하여 시각화
    
    Args:
        image_paths: 이미지 경로 리스트
        labels: 라벨 리스트
        class_to_idx: 클래스명-인덱스 매핑 딕셔너리
        num_samples: 시각화할 샘플 개수 (기본값: 5)
    """
    # 인덱스-클래스명 매핑
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    # 랜덤 인덱스 선택
    random_indices = random.sample(range(len(image_paths)), min(num_samples, len(image_paths)))
    
    # 플롯 생성
    fig, axes = plt.subplots(1, len(random_indices), figsize=(4 * len(random_indices), 4))
    if len(random_indices) == 1:
        axes = [axes]
    
    for idx, img_idx in enumerate(random_indices):
        # 이미지 로드
        img_path = image_paths[img_idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 클래스명 가져오기
        label_idx = labels[img_idx]
        class_name = idx_to_class[label_idx]
        
        # 시각화
        axes[idx].imshow(image)
        axes[idx].set_title(f'Class: {class_name}\n({os.path.basename(img_path)})', fontsize=10)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# 데이터가 로드된 경우 랜덤 샘플 시각화
if 'train_paths' in locals() and len(train_paths) > 0:
    print(f"\n{'='*60}")
    print("데이터셋에서 랜덤으로 5개 샘플 추출 (원본 이미지)")
    print(f"{'='*60}\n")
    visualize_random_samples(train_paths, train_labels, class_to_idx, num_samples=5)
    
    # 클래스별 분포 출력
    print(f"\n{'='*60}")
    print("클래스별 이미지 개수:")
    print(f"{'='*60}")
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    class_counts = {}
    for label in train_labels:
        class_name = idx_to_class[label]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1
    
    for class_name in sorted(class_counts.keys()):
        print(f"  {class_name}: {class_counts[class_name]:4d} images")
    print(f"{'='*60}\n")

## 8-1. 데이터셋 확인 (랜덤 샘플 5개 시각화)

로드한 데이터가 올바른지 확인하기 위해 각 클래스에서 랜덤으로 샘플을 추출하여 확인합니다.

## 8-2. 추가 데이터 분석 (EDA)

데이터의 특성을 더 자세히 파악하기 위한 시각화입니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

def visualize_class_distribution(labels, class_to_idx):
    """Visualize the distribution of classes in the dataset"""
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    class_names = [idx_to_class[label] for label in labels]
    class_counts = Counter(class_names)
    
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(classes, counts)
    plt.xlabel('Class')
    plt.ylabel('Number of Images')
    plt.title('Class Distribution in Training Dataset')
    plt.xticks(rotation=45, ha='right')
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print("\nClass distribution:")
    for class_name, count in sorted(class_counts.items()):
        print(f"{class_name}: {count} images")

def analyze_image_resolutions(image_paths, num_samples=None):
    """Analyze and visualize image resolution distribution"""
    from PIL import Image
    
    if num_samples:
        sample_paths = np.random.choice(image_paths, min(num_samples, len(image_paths)), replace=False)
    else:
        sample_paths = image_paths
    
    widths = []
    heights = []
    
    for img_path in sample_paths:
        try:
            with Image.open(img_path) as img:
                w, h = img.size
                widths.append(w)
                heights.append(h)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    axes[0].hist(widths, bins=30, edgecolor='black')
    axes[0].set_xlabel('Width (pixels)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Image Width Distribution')
    axes[0].axvline(np.mean(widths), color='r', linestyle='--', label=f'Mean: {np.mean(widths):.0f}')
    axes[0].legend()
    
    axes[1].hist(heights, bins=30, edgecolor='black')
    axes[1].set_xlabel('Height (pixels)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Image Height Distribution')
    axes[1].axvline(np.mean(heights), color='r', linestyle='--', label=f'Mean: {np.mean(heights):.0f}')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nResolution Statistics (from {len(widths)} images):")
    print(f"Width  - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.0f}")
    print(f"Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.0f}")

def visualize_class_grid(image_paths, labels, class_to_idx, samples_per_class=3):
    """Visualize random samples from each class in a grid"""
    from PIL import Image
    
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    class_images = {class_name: [] for class_name in class_to_idx.keys()}
    for img_path, label in zip(image_paths, labels):
        class_name = idx_to_class[label]
        class_images[class_name].append(img_path)
    
    num_classes = len(class_to_idx)
    fig, axes = plt.subplots(num_classes, samples_per_class, figsize=(samples_per_class * 3, num_classes * 3))
    
    if num_classes == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (class_name, img_paths) in enumerate(sorted(class_images.items())):
        sample_paths = np.random.choice(img_paths, min(samples_per_class, len(img_paths)), replace=False)
        
        for col, img_path in enumerate(sample_paths):
            try:
                img = Image.open(img_path)
                axes[idx, col].imshow(img)
                axes[idx, col].axis('off')
                if col == 0:
                    axes[idx, col].set_ylabel(class_name, rotation=0, ha='right', va='center', fontsize=12)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
        
        for col in range(len(sample_paths), samples_per_class):
            axes[idx, col].axis('off')
    
    plt.suptitle('Random Samples from Each Class', fontsize=16, y=0.995)
    plt.tight_layout()
    plt.show()

if 'train_paths' in locals() and len(train_paths) > 0:
    print("=" * 50)
    print("EXPLORATORY DATA ANALYSIS")
    print("=" * 50)
    
    print("\n1. Class Distribution Analysis")
    print("-" * 50)
    visualize_class_distribution(train_labels, class_to_idx)
    
    print("\n2. Image Resolution Analysis")
    print("-" * 50)
    analyze_image_resolutions(train_paths, num_samples=500)
    
    print("\n3. Visual Sample Grid")
    print("-" * 50)
    visualize_class_grid(train_paths, train_labels, class_to_idx, samples_per_class=3)

In [ ]:
# K-Fold 설정n_splits = 3skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=Config.seed)print(f"\n{'='*60}")print(f"3-Fold Cross Validation Ensemble Training")print(f"{'='*60}\n")

In [ ]:
# 데이터셋 및 데이터로더 생성if 'train_paths' in locals():    train_dataset = AlbumentationsDataset(train_paths, train_labels, train_transform)    val_dataset = AlbumentationsDataset(val_paths, val_labels, val_transform)        train_loader = DataLoader(        train_dataset,         batch_size=Config.batch_size,         shuffle=True,         num_workers=Config.num_workers    )        val_loader = DataLoader(        val_dataset,         batch_size=Config.batch_size,         shuffle=False,         num_workers=Config.num_workers    )

## 9. 증강 결과 시각화 (선택사항)

증강이 어떻게 적용되는지 확인해보세요!

In [ ]:
import matplotlib.pyplot as pltdef visualize_augmentations(dataset, idx=0, samples=5):    """    데이터셋의 증강 결과를 시각화        Args:        dataset: AlbumentationsDataset 객체        idx: 시각화할 이미지의 인덱스        samples: 생성할 증강 샘플 수    """    # 원본 이미지 로드    img_path = dataset.image_paths[idx]    original_image = cv2.imread(img_path)    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)        # 플롯 생성    fig, axes = plt.subplots(1, samples + 1, figsize=(4 * (samples + 1), 4))        # 원본 이미지    axes[0].imshow(original_image)    axes[0].set_title('Original', fontsize=12)    axes[0].axis('off')        # 증강된 이미지들    for i in range(samples):        augmented = dataset.transform(image=original_image)        aug_image = augmented['image']                # 텐서를 numpy로 변환하고 정규화 해제        if isinstance(aug_image, torch.Tensor):            aug_image = aug_image.permute(1, 2, 0).numpy()            # 정규화 해제            mean = np.array([0.485, 0.456, 0.406])            std = np.array([0.229, 0.224, 0.225])            aug_image = std * aug_image + mean            aug_image = np.clip(aug_image, 0, 1)                axes[i + 1].imshow(aug_image)        axes[i + 1].set_title(f'Augmented {i+1}', fontsize=12)        axes[i + 1].axis('off')        plt.tight_layout()    plt.show()# 증강 시각화 (데이터가 로드된 경우)if 'train_dataset' in locals() and len(train_dataset) > 0:    print(f"Visualizing augmentations with level: {Config.augmentation_level}")    visualize_augmentations(train_dataset, idx=0, samples=4)

## 10. WandB Run 초기화 (학습 시작 전)

In [ ]:
def get_next_experiment_number(project_name, prefix, entity=None):    """WandB에서 기존 실험들을 확인하고 다음 번호를 반환"""    try:        api = wandb.Api()        # 프로젝트의 모든 run 가져오기        if entity:            runs = api.runs(f"{entity}/{project_name}")        else:            runs = api.runs(project_name)                # prefix로 시작하는 run들의 번호 추출        numbers = []        for run in runs:            if run.name.startswith(prefix):                try:                    # 'prefix_123' 형태에서 123 추출                    num = int(run.name.split('_')[-1])                    numbers.append(num)                except:                    continue                # 가장 큰 번호 + 1 반환        next_num = max(numbers) + 1 if numbers else 1        return next_num    except:        # API 접근 실패시 001부터 시작        return 1# WandB Run 초기화if Config.use_wandb:    # experiment_prefix가 None이면 모델명 사용 (자동)    if Config.experiment_prefix is None:        actual_prefix = f"{Config.model_name}_alb_{Config.augmentation_level}"    else:        actual_prefix = Config.experiment_prefix        # 실험명 자동 생성    if Config.experiment_name is None:        exp_num = get_next_experiment_number(            Config.wandb_project,             actual_prefix,            Config.wandb_entity        )        Config.experiment_name = f"{actual_prefix}_{exp_num:03d}"        run = wandb.init(        project=Config.wandb_project,        entity=Config.wandb_entity,        name=Config.experiment_name,        config={            "model_name": Config.model_name,            "num_classes": Config.num_classes,            "img_size": Config.img_size,            "epochs": Config.epochs,            "batch_size": Config.batch_size,            "learning_rate": Config.learning_rate,            "weight_decay": Config.weight_decay,            "optimizer": "AdamW",            "scheduler": "CosineAnnealingLR",            "val_ratio": Config.val_ratio,            "seed": Config.seed,            "augmentation": "albumentations",            "augmentation_level": Config.augmentation_level,        }    )    print(f"\n{'='*60}")    print(f"WandB Run initialized: {run.name}")    print(f"WandB URL: {run.url}")    print(f"{'='*60}\n")else:    print("WandB is disabled")

## 11. 모델 정의

In [ ]:
class DocumentClassifier(nn.Module):    def __init__(self, model_name, num_classes, pretrained=True):        super(DocumentClassifier, self).__init__()        self.model = timm.create_model(model_name, pretrained=pretrained)                # 모델의 classifier 부분 수정        if 'efficientnet' in model_name:            in_features = self.model.classifier.in_features            self.model.classifier = nn.Linear(in_features, num_classes)        elif 'resnet' in model_name:            in_features = self.model.fc.in_features            self.model.fc = nn.Linear(in_features, num_classes)        elif 'vit' in model_name:            in_features = self.model.head.in_features            self.model.head = nn.Linear(in_features, num_classes)        def forward(self, x):        return self.model(x)# 모델 생성model = DocumentClassifier(    model_name=Config.model_name,     num_classes=Config.num_classes,     pretrained=True).to(device)print(f"Model: {Config.model_name}")print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")# WandB에 모델 아키텍처 로깅if Config.use_wandb:    wandb.watch(model, log='all', log_freq=100)

## 12. 손실 함수 및 옵티마이저

In [ ]:
criterion = nn.CrossEntropyLoss()optimizer = optim.AdamW(model.parameters(), lr=Config.learning_rate, weight_decay=Config.weight_decay)scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.epochs, eta_min=1e-6)

## 13. 학습 및 검증 함수

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch):    model.train()    running_loss = 0.0    correct = 0    total = 0        pbar = tqdm(train_loader, desc='Training')    for batch_idx, (images, labels) in enumerate(pbar):        images, labels = images.to(device), labels.to(device)                optimizer.zero_grad()        outputs = model(images)        loss = criterion(outputs, labels)        loss.backward()        optimizer.step()                running_loss += loss.item()        _, predicted = outputs.max(1)        total += labels.size(0)        correct += predicted.eq(labels).sum().item()                # 배치별 메트릭 계산        batch_loss = running_loss / (batch_idx + 1)        batch_acc = 100. * correct / total                pbar.set_postfix({            'loss': batch_loss,            'acc': batch_acc        })                # WandB 로깅 (매 배치마다)        if Config.use_wandb:            wandb.log({                'train/batch_loss': loss.item(),                'train/batch_acc': 100. * predicted.eq(labels).sum().item() / labels.size(0),                'train/step': epoch * len(train_loader) + batch_idx            })        epoch_loss = running_loss / len(train_loader)    epoch_acc = 100. * correct / total    return epoch_loss, epoch_acc

In [ ]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(val_loader)
    
    # Macro F1 Score 계산
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    
    # 클래스별 F1 Score 계산
    per_class_f1 = f1_score(all_labels, all_preds, average=None)
    
    # Confusion Matrix 계산
    cm = confusion_matrix(all_labels, all_preds)
    
    return epoch_loss, macro_f1, per_class_f1, cm, all_preds, all_labels

## 14. 학습 실행

In [ ]:
# K-Fold 앙상블 학습 시작fold_models = []  # 각 fold의 모델을 저장fold_results = []  # 각 fold의 결과를 저장# 전체 히스토리 저장용all_fold_history = {    'train_loss': [],    'train_acc': [],    'val_loss': [],    'val_f1': []}for fold, (train_idx, val_idx) in enumerate(skf.split(train_paths, train_labels)):    print(f"\n{'='*60}")    print(f"Fold {fold+1}/{n_splits}")    print(f"{'='*60}\n")        # 현재 fold의 train/val split    fold_train_paths = [train_paths[i] for i in train_idx]    fold_val_paths = [train_paths[i] for i in val_idx]    fold_train_labels = [train_labels[i] for i in train_idx]    fold_val_labels = [train_labels[i] for i in val_idx]        print(f"Fold {fold+1} - Train size: {len(fold_train_paths)}")    print(f"Fold {fold+1} - Val size: {len(fold_val_paths)}")        # 데이터셋 및 데이터로더 생성    fold_train_dataset = AlbumentationDataset(fold_train_paths, fold_train_labels, train_transform)    fold_val_dataset = AlbumentationDataset(fold_val_paths, fold_val_labels, val_transform)        train_loader = DataLoader(        fold_train_dataset,        batch_size=Config.batch_size,        shuffle=True,        num_workers=Config.num_workers    )        val_loader = DataLoader(        fold_val_dataset,        batch_size=Config.batch_size,        shuffle=False,        num_workers=Config.num_workers    )        # 새 모델 생성 (각 fold마다 새로운 모델)    model = DocumentClassifier(        model_name=Config.model_name,        num_classes=Config.num_classes,        pretrained=True    ).to(device)        criterion = nn.CrossEntropyLoss()    optimizer = optim.AdamW(model.parameters(), lr=Config.learning_rate, weight_decay=Config.weight_decay)    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.epochs, eta_min=1e-6)        # WandB에 모델 아키텍처 로깅 (첫 번째 fold에만)    if Config.use_wandb and fold == 0:        wandb.watch(model, log='all', log_freq=100)        # Fold별 학습    best_f1 = 0.0    patience_counter = 0    history = {        'train_loss': [],        'train_acc': [],        'val_loss': [],        'val_f1': []    }        # 구글 드라이브 저장 경로 생성    if Config.save_to_drive:        os.makedirs(Config.drive_model_dir, exist_ok=True)        for epoch in range(Config.epochs):        print(f"\nFold {fold+1} - Epoch {epoch+1}/{Config.epochs}")        print("-" * 50)                # 학습        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch)                # 검증        val_loss, val_f1, per_class_f1, cm, val_preds, val_labels_batch = validate(model, val_loader, criterion, device)                # 스케줄러 업데이트        scheduler.step()        current_lr = optimizer.param_groups[0]['lr']                # 결과 저장        history['train_loss'].append(train_loss)        history['train_acc'].append(train_acc)        history['val_loss'].append(val_loss)        history['val_f1'].append(val_f1)                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")        print(f"Val Loss: {val_loss:.4f}, Val Macro F1: {val_f1:.4f}")        print(f"Learning Rate: {current_lr:.6f}")                # WandB 로깅        if Config.use_wandb:            log_dict = {                'fold': fold + 1,                'epoch': epoch + 1,                f'fold{fold+1}/train_loss': train_loss,                f'fold{fold+1}/train_acc': train_acc,                f'fold{fold+1}/val_loss': val_loss,                f'fold{fold+1}/val_f1': val_f1,                f'fold{fold+1}/learning_rate': current_lr,                f'fold{fold+1}/patience_counter': patience_counter,            }                        # 클래스별 F1 Score            if 'class_to_idx' in locals():                idx_to_class = {v: k for k, v in class_to_idx.items()}                for idx, f1 in enumerate(per_class_f1):                    class_name = idx_to_class.get(idx, f'class_{idx}')                    log_dict[f'fold{fold+1}/f1_{class_name}'] = f1                        wandb.log(log_dict)                # Early Stopping 체크 및 베스트 모델 저장        if val_f1 > best_f1 + Config.early_stopping_min_delta:            best_f1 = val_f1            patience_counter = 0                        checkpoint = {                'fold': fold,                'epoch': epoch,                'model_state_dict': model.state_dict(),                'optimizer_state_dict': optimizer.state_dict(),                'best_f1': best_f1,            }                        # 로컬에 저장 (fold 번호 포함)            local_model_path = f'best_model_alb_{Config.augmentation_level}_fold{fold+1}.pth'            torch.save(checkpoint, local_model_path)            print(f"✓ Fold {fold+1} best model saved locally! (F1: {best_f1:.4f})")                        # 구글 드라이브에 저장            if Config.save_to_drive:                if Config.use_wandb and Config.experiment_name:                    drive_model_path = f"{Config.drive_model_dir}/{Config.experiment_name}_fold{fold+1}_f1_{best_f1:.4f}.pth"                else:                    drive_model_path = f"{Config.drive_model_dir}/best_model_alb_{Config.augmentation_level}_fold{fold+1}_f1_{best_f1:.4f}.pth"                                torch.save(checkpoint, drive_model_path)                if IS_COLAB:                    print(f"✓ Fold {fold+1} best model saved to Google Drive: {drive_model_path}")                else:                    print(f"✓ Fold {fold+1} best model saved locally: {drive_model_path}")                        # WandB에 베스트 모델 저장            if Config.use_wandb:                artifact = wandb.Artifact(                    name=f'model-fold{fold+1}-{run.id}',                    type='model',                    description=f'Fold {fold+1} best model with F1: {best_f1:.4f}',                    metadata={                        'fold': fold + 1,                        'epoch': epoch + 1,                        'val_f1': val_f1,                        'val_loss': val_loss,                        'augmentation_level': Config.augmentation_level,                    }                )                artifact.add_file(local_model_path)                wandb.log_artifact(artifact)        else:            patience_counter += 1            print(f"⚠ No improvement. Patience: {patience_counter}/{Config.early_stopping_patience}")                        if patience_counter >= Config.early_stopping_patience:                print(f"\n{'='*60}")                print(f"Fold {fold+1} - Early Stopping triggered at epoch {epoch+1}")                print(f"Fold {fold+1} - Best Validation Macro F1: {best_f1:.4f}")                print(f"{'='*60}")                break        # Fold 결과 저장    fold_results.append({        'fold': fold + 1,        'best_f1': best_f1,        'history': history,        'final_epoch': epoch + 1    })        # 전체 히스토리에 추가    all_fold_history['train_loss'].extend(history['train_loss'])    all_fold_history['train_acc'].extend(history['train_acc'])    all_fold_history['val_loss'].extend(history['val_loss'])    all_fold_history['val_f1'].extend(history['val_f1'])        print(f"\n{'='*60}")    print(f"Fold {fold+1} completed!")    print(f"Fold {fold+1} - Best Validation Macro F1: {best_f1:.4f}")    print(f"Fold {fold+1} - Total epochs: {epoch+1}")    print(f"{'='*60}\n")# 전체 학습 완료 후 결과 출력print(f"\n{'='*60}")print(f"3-Fold Cross Validation Training Completed!")print(f"{'='*60}")for result in fold_results:    print(f"Fold {result['fold']}: Best F1 = {result['best_f1']:.4f} (Epochs: {result['final_epoch']})")avg_f1 = sum([r['best_f1'] for r in fold_results]) / len(fold_results)print(f"\nAverage F1 across folds: {avg_f1:.4f}")print(f"{'='*60}")# 최종 모델 경로 출력print(f"\n모델 저장 위치:")for fold in range(n_splits):    print(f"  - Fold {fold+1} 로컬: best_model_alb_{Config.augmentation_level}_fold{fold+1}.pth")if Config.save_to_drive:    print(f"  - 드라이브: {Config.drive_model_dir}/")# WandB에 최종 메트릭 로깅if Config.use_wandb:    wandb.log({        'final/avg_f1': avg_f1,        'final/best_fold': max(fold_results, key=lambda x: x['best_f1'])['fold'],        'final/best_f1': max([r['best_f1'] for r in fold_results]),    })    wandb.finish()# history 변수를 all_fold_history로 설정 (이후 시각화를 위해)history = all_fold_history

## 15. 학습 결과 시각화

In [ ]:
import matplotlib.pyplot as pltfig, axes = plt.subplots(1, 2, figsize=(15, 5))# Loss 그래프axes[0].plot(history['train_loss'], label='Train Loss')axes[0].plot(history['val_loss'], label='Val Loss')axes[0].set_xlabel('Epoch')axes[0].set_ylabel('Loss')axes[0].set_title(f'Training and Validation Loss (Albumentations {Config.augmentation_level})')axes[0].legend()axes[0].grid(True)# F1 Score 그래프axes[1].plot(history['val_f1'], label='Val Macro F1', color='orange')axes[1].set_xlabel('Epoch')axes[1].set_ylabel('Macro F1 Score')axes[1].set_title(f'Validation Macro F1 Score (Albumentations {Config.augmentation_level})')axes[1].legend()axes[1].grid(True)plt.tight_layout()plt.show()

## 💾 실험 결과 백업 (노트북 사본 저장)

학습이 끝난 후 현재 노트북을 **모델명_F1스코어.ipynb** 형식으로 자동 저장합니다.

**저장 예시:**
- `efficientnet_b0_f1_0.8234.ipynb`
- `efficientnet_b1_f1_0.8567.ipynb`
- `convnext_tiny_f1_0.8892.ipynb`

**장점:**
- 각 실험 결과를 쉽게 비교
- 나중에 최고 성능 노트북 찾기 쉬움
- 실험 히스토리 자동 관리

In [ ]:
import shutilfrom datetime import datetimedef save_notebook_with_score(model_name, f1_score, notebook_name='notebook.ipynb',                              save_dir='./experiments'):    """    현재 노트북을 모델명과 F1 스코어를 포함한 이름으로 저장        Args:        model_name: 모델명 (예: 'efficientnet_b0')        f1_score: F1 스코어 (예: 0.8234)        notebook_name: 현재 노트북 파일명 (Colab: 자동 감지)        save_dir: 저장할 디렉토리        Returns:        저장된 파일 경로    """    import os        # Colab에서 현재 노트북 이름 자동 감지    try:        from google.colab import drive        # Colab 환경에서는 ipynb 파일명을 알 수 없으므로 기본값 사용        # 수동으로 지정하거나 마운트된 드라이브에서 확인 필요        pass    except:        pass        # 저장 디렉토리 생성    os.makedirs(save_dir, exist_ok=True)        # 타임스탬프 추가    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')        # 새 파일명 생성    new_filename = f"{model_name}_f1_{f1_score:.4f}_{timestamp}.ipynb"    save_path = os.path.join(save_dir, new_filename)        # 현재 노트북 복사    try:        # Colab 환경에서는 직접 복사가 어려우므로 드라이브에 저장        if os.path.exists(notebook_name):            shutil.copy2(notebook_name, save_path)            print(f"✓ 노트북 저장 완료: {save_path}")        else:            # Colab에서는 구글 드라이브에 수동 저장 안내            print(f"\n{'='*70}")            print("Colab 환경에서는 다음 방법으로 노트북을 저장하세요:")            print(f"{'='*70}")            print(f"1. 상단 메뉴: 파일 → 드라이브에 사본 저장")            print(f"2. 파일명을 다음과 같이 변경:")            print(f"   {new_filename}")            print(f"{'='*70}\n")                        # 구글 드라이브에 자동 저장 (마운트되어 있는 경우)            if Config.save_to_drive and os.path.exists('/content/drive'):                drive_notebook_dir = '/content/drive/MyDrive/document_classification/notebooks'                os.makedirs(drive_notebook_dir, exist_ok=True)                drive_path = os.path.join(drive_notebook_dir, new_filename)                                print(f"💡 추천 저장 경로:")                print(f"   {drive_path}")                print(f"\n   → 위 경로에 수동으로 저장하세요!")                                return drive_path    except Exception as e:        print(f"✗ 저장 실패: {e}")        return None        return save_pathdef save_experiment_summary(model_name, f1_score, history, config_dict,                            save_path='experiment_summary.txt'):    """    실험 결과 요약 텍스트 파일 저장        Args:        model_name: 모델명        f1_score: 최고 F1 스코어        history: 학습 히스토리        config_dict: 설정 정보 딕셔너리        save_path: 저장 경로    """    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')        summary = f"""{'='*70}실험 결과 요약{'='*70}실험 일시: {timestamp}모델명: {model_name}최고 Validation F1: {f1_score:.4f}{'='*70}학습 설정{'='*70}"""        for key, value in config_dict.items():        summary += f"{key}: {value}\n"        summary += f"""{'='*70}학습 히스토리{'='*70}Epoch | Train Loss | Val Loss | Val F1------+------------+----------+--------"""        for i in range(len(history['train_loss'])):        summary += f"{i+1:5d} | {history['train_loss'][i]:10.4f} | {history['val_loss'][i]:8.4f} | {history['val_f1'][i]:6.4f}\n"        summary += f"""{'='*70}최종 결과{'='*70}Best Validation F1: {f1_score:.4f}Total Epochs: {len(history['train_loss'])}Final Train Loss: {history['train_loss'][-1]:.4f}Final Val Loss: {history['val_loss'][-1]:.4f}{'='*70}"""        with open(save_path, 'w', encoding='utf-8') as f:        f.write(summary)        print(f"✓ 실험 요약 저장: {save_path}")        # 구글 드라이브에도 저장    if Config.save_to_drive and os.path.exists('/content/drive'):        drive_summary_dir = '/content/drive/MyDrive/document_classification/summaries'        os.makedirs(drive_summary_dir, exist_ok=True)        drive_summary_path = os.path.join(drive_summary_dir,                                          f"{model_name}_f1_{f1_score:.4f}_summary.txt")        shutil.copy2(save_path, drive_summary_path)        print(f"✓ 구글 드라이브에도 저장: {drive_summary_path}")# 학습 완료 후 자동 저장if 'best_f1' in locals() and 'history' in locals():    print(f"\n{'='*70}")    print("실험 결과 자동 백업")    print(f"{'='*70}\n")        # 1. 실험 요약 텍스트 파일 저장    config_dict = {        'Model': Config.model_name,        'Image Size': Config.img_size,        'Batch Size': Config.batch_size,        'Learning Rate': Config.learning_rate,        'Epochs': Config.epochs,        'Augmentation': getattr(Config, 'augmentation_level', 'N/A'),    }        save_experiment_summary(        model_name=Config.model_name,        f1_score=best_f1,        history=history,        config_dict=config_dict,        save_path=f'{Config.model_name}_f1_{best_f1:.4f}_summary.txt'    )        # 2. 노트북 사본 저장 안내    print(f"\n📓 노트북 저장 안내:")    save_notebook_with_score(        model_name=Config.model_name,        f1_score=best_f1,        notebook_name='current_notebook.ipynb'  # Colab에서는 자동 감지 불가    )        print(f"\n{'='*70}")    print("백업 완료! 🎉")    print(f"{'='*70}")else:    print("⚠️ 학습이 아직 완료되지 않았습니다.")

## 14. 틀린 이미지 분석 (Error Analysis)

학습이 끝난 후 validation set에서 틀리게 예측한 이미지들을 확인하여 모델의 약점을 파악할 수 있습니다.

**분석 내용:**
- 가장 확신있게 틀린 이미지 (모델이 확신했는데 틀린 경우)
- 클래스별 주요 오분류 패턴
- 낮은 확신도로 맞춘 샘플 (불확실한 경우)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def analyze_misclassified_images(model, val_dataset, val_paths, val_labels, class_to_idx, 
                                 device, num_samples=12, save_path=None):
    """
    틀린 이미지들을 시각화하여 모델의 약점 분석
    
    Args:
        model: 학습된 모델
        val_dataset: Validation 데이터셋
        val_paths: Validation 이미지 경로 리스트
        val_labels: Validation 라벨 리스트
        class_to_idx: 클래스명-인덱스 매핑
        device: 디바이스
        num_samples: 시각화할 샘플 개수
        save_path: 이미지 저장 경로 (None이면 저장 안 함)
    """
    model.eval()
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    # 틀린 이미지 정보 수집
    misclassified = []
    
    with torch.no_grad():
        for i in range(len(val_dataset)):
            image, true_label = val_dataset[i]
            image_tensor = image.unsqueeze(0).to(device)
            
            # 예측
            output = model(image_tensor)
            _, predicted = output.max(1)
            pred_label = predicted.item()
            
            # 틀린 경우만 저장
            if pred_label != true_label:
                # 확률 계산
                probs = torch.softmax(output, dim=1)[0]
                confidence = probs[pred_label].item()
                true_confidence = probs[true_label].item()
                
                misclassified.append({
                    'index': i,
                    'path': val_paths[i],
                    'true_label': true_label,
                    'pred_label': pred_label,
                    'true_class': idx_to_class[true_label],
                    'pred_class': idx_to_class[pred_label],
                    'confidence': confidence,
                    'true_confidence': true_confidence
                })
    
    # 결과 출력
    total_samples = len(val_dataset)
    num_errors = len(misclassified)
    accuracy = (total_samples - num_errors) / total_samples * 100
    
    print(f"\n{'='*70}")
    print(f"Validation Set 에러 분석")
    print(f"{'='*70}")
    print(f"전체 샘플: {total_samples}개")
    print(f"틀린 샘플: {num_errors}개")
    print(f"정확도: {accuracy:.2f}%")
    print(f"{'='*70}\n")
    
    if num_errors == 0:
        print("✓ 완벽! 모든 샘플을 정확히 예측했습니다!")
        return
    
    # 클래스별 에러 분석
    class_errors = {}
    for item in misclassified:
        true_class = item['true_class']
        pred_class = item['pred_class']
        
        if true_class not in class_errors:
            class_errors[true_class] = {}
        
        if pred_class not in class_errors[true_class]:
            class_errors[true_class][pred_class] = 0
        
        class_errors[true_class][pred_class] += 1
    
    print("📊 클래스별 주요 오분류:")
    print(f"{'='*70}")
    for true_class in sorted(class_errors.keys()):
        error_pairs = sorted(class_errors[true_class].items(), 
                           key=lambda x: x[1], reverse=True)
        top_errors = error_pairs[:3]  # 상위 3개만
        
        print(f"\n{true_class} (실제):")
        for pred_class, count in top_errors:
            print(f"  → {pred_class} (예측): {count}회")
    print(f"\n{'='*70}\n")
    
    # 시각화할 샘플 선택 (confidence가 높은 순서대로)
    # confidence가 높은데 틀린 경우 = 모델이 확신했는데 틀린 케이스
    misclassified_sorted = sorted(misclassified, 
                                 key=lambda x: x['confidence'], 
                                 reverse=True)
    
    samples_to_show = min(num_samples, len(misclassified_sorted))
    
    # 그리드 레이아웃 계산
    cols = 4
    rows = (samples_to_show + cols - 1) // cols
    
    # 시각화
    fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx in range(rows * cols):
        row = idx // cols
        col = idx % cols
        ax = axes[row, col]
        
        if idx < samples_to_show:
            item = misclassified_sorted[idx]
            
            # 이미지 로드
            img = Image.open(item['path']).convert('RGB')
            
            # 시각화
            ax.imshow(img)
            
            # 제목 설정 (여러 줄)
            title = f"실제: {item['true_class']}\n"
            title += f"예측: {item['pred_class']}\n"
            title += f"확신도: {item['confidence']:.1%}"
            
            # 색상 설정 (확신도가 높을수록 빨간색)
            title_color = 'red' if item['confidence'] > 0.7 else 'orange'
            
            ax.set_title(title, fontsize=9, color=title_color, fontweight='bold')
            ax.axis('off')
        else:
            ax.axis('off')
    
    plt.suptitle(f'가장 확신있게 틀린 이미지 Top {samples_to_show}', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✓ 분석 결과 저장: {save_path}")
    
    plt.show()
    
    # 추가 분석: 낮은 confidence로 맞춘 경우
    correct_low_conf = []
    with torch.no_grad():
        for i in range(len(val_dataset)):
            image, true_label = val_dataset[i]
            image_tensor = image.unsqueeze(0).to(device)
            
            output = model(image_tensor)
            probs = torch.softmax(output, dim=1)[0]
            _, predicted = output.max(1)
            pred_label = predicted.item()
            
            if pred_label == true_label:
                confidence = probs[pred_label].item()
                if confidence < 0.6:  # 확신도가 낮은데 맞춘 경우
                    correct_low_conf.append({
                        'index': i,
                        'path': val_paths[i],
                        'true_label': true_label,
                        'true_class': idx_to_class[true_label],
                        'confidence': confidence
                    })
    
    if correct_low_conf:
        print(f"\n{'='*70}")
        print(f"⚠️ 낮은 확신도로 맞춘 샘플: {len(correct_low_conf)}개")
        print(f"{'='*70}")
        print("모델이 불확실해하는 샘플들입니다. 추가 학습이나 증강이 필요할 수 있습니다.\n")


# 에러 분석 실행 (베스트 모델 로드 후)
if 'val_dataset' in locals() and 'val_paths' in locals():
    print("\n🔍 Validation Set 에러 분석을 시작합니다...")
    analyze_misclassified_images(
        model=model,
        val_dataset=val_dataset,
        val_paths=val_paths,
        val_labels=val_labels,
        class_to_idx=class_to_idx,
        device=device,
        num_samples=12,  # 12개 샘플 표시
        save_path='error_analysis.png'  # 결과 저장
    )
else:
    print("⚠️ Validation 데이터가 없습니다. 데이터를 먼저 로드해주세요.")

## 17. 테스트 데이터 추론

In [ ]:
# 테스트 데이터 로드def load_test_data(test_dir):    test_paths = []    for img_name in sorted(os.listdir(test_dir)):        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):            test_paths.append(os.path.join(test_dir, img_name))    return test_pathsif os.path.exists(Config.test_dir):    test_paths = load_test_data(Config.test_dir)    print(f"Total test images: {len(test_paths)}")else:    print(f"Warning: {Config.test_dir} does not exist!")    test_paths = []

In [ ]:
# 테스트 데이터셋 및 로더 생성if test_paths:    test_dataset = AlbumentationsDataset(test_paths, labels=None, transform=val_transform)    test_loader = DataLoader(        test_dataset,         batch_size=Config.batch_size,         shuffle=False,         num_workers=Config.num_workers    )

In [ ]:
# 앙상블 예측 (3개 fold 모델 평균)print(f"\n{'='*60}")print("Ensemble Prediction with 3 Fold Models")print(f"{'='*60}\n")# 모든 fold 모델 로드fold_models = []for fold in range(n_splits):    model_fold = DocumentClassifier(        model_name=Config.model_name,        num_classes=Config.num_classes,        pretrained=False    )    checkpoint_path = f"best_model_alb_{Config.augmentation_level}_fold{fold+1}.pth"    checkpoint = torch.load(checkpoint_path, map_location=device)    model_fold.load_state_dict(checkpoint['model_state_dict'])    model_fold = model_fold.to(device)    model_fold.eval()    fold_models.append(model_fold)    print(f"✓ Loaded fold {fold+1} model (F1: {checkpoint['best_f1']:.4f})")# 앙상블 예측ensemble_preds = []with torch.no_grad():    for images in tqdm(test_loader, desc="Ensemble Inference"):        if isinstance(images, tuple):            images = images[0]        images = images.to(device)                # 각 fold 모델의 예측을 평균        fold_outputs = []        for model_fold in fold_models:            outputs = model_fold(images)            fold_outputs.append(torch.softmax(outputs, dim=1))                # 평균 예측        avg_output = torch.stack(fold_outputs).mean(dim=0)        preds = avg_output.argmax(dim=1)        ensemble_preds.extend(preds.cpu().numpy())print(f"\n✓ Ensemble prediction completed for {len(ensemble_preds)} test images")# predictions 변수를 ensemble_preds로 업데이트predictions = ensemble_preds

## 18. 제출 파일 생성

In [ ]:
# 제출 파일 생성 (형식은 대회 규정에 맞게 수정)if test_paths and predictions:    # 클래스 인덱스를 클래스 이름으로 변환    idx_to_class = {v: k for k, v in class_to_idx.items()}        submission = pd.DataFrame({        'image': [os.path.basename(path) for path in test_paths],        'label': [idx_to_class[pred] for pred in predictions]    })        submission_filename = f'submission_alb_{Config.augmentation_level}.csv'    submission.to_csv(submission_filename, index=False)    print(f"\nSubmission file saved: {submission_filename}")    print(submission.head(10))

## 19. 고급 팁 및 추가 개선 아이디어

### 🔬 커스텀 증강 만들기

기본 제공되는 3가지 레벨 외에 직접 커스텀 증강을 만들 수 있습니다:

```python
# 예시: 문서 스캔 특화 증강
custom_transform = A.Compose([
    A.Resize(CFG.img_size, CFG.img_size),
    
    # 스캔 시 발생하는 노이즈
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
        A.ISONoise(p=1.0),
    ], p=0.3),
    
    # 구겨진 문서 효과
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    
    # 팩스/복사기 블러
    A.Blur(blur_limit=3, p=0.2),
    
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
```

---

### 🎯 Test Time Augmentation (TTA)

추론 시에도 증강을 적용하여 **앙상블 효과**를 얻을 수 있습니다:

```python
def predict_with_tta(model, image, transforms, n_augmentations=5):
    """
    TTA를 적용한 예측 함수
    같은 이미지를 여러 번 증강하고 예측을 평균내어 더 안정적인 결과 획득
    """
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for _ in range(n_augmentations):
            augmented = transforms(image=image)
            img_tensor = augmented['image'].unsqueeze(0).to(device)
            output = model(img_tensor)
            predictions.append(output.softmax(dim=1))
    
    # 평균 예측
    avg_prediction = torch.stack(predictions).mean(dim=0)
    return avg_prediction.argmax(dim=1).item()
```

**TTA 사용 시 주의사항:**
- 추론 시간이 n_augmentations배 증가
- 일반적으로 0.5-2% 성능 향상
- 최종 제출 시에만 사용 권장

---

### 📈 성능 향상을 위한 추가 아이디어

#### 1. **증강 파라미터 튜닝**
```python
# WandB Sweep으로 최적 증강 강도 찾기
sweep_config = {
    'method': 'bayes',
    'parameters': {
        'augmentation_level': {
            'values': ['light', 'medium', 'heavy']
        },
        'learning_rate': {
            'min': 1e-5,
            'max': 1e-3
        }
    }
}
```

#### 2. **MixUp / CutMix**
```python
# Albumentations의 MixUp transform
A.MixUp(alpha=0.2, p=0.5)
```

#### 3. **클래스별 맞춤 증강**
```python
# 불균형 데이터셋의 경우 소수 클래스에 더 강한 증강 적용
if label in minority_classes:
    transform = heavy_transform
else:
    transform = medium_transform
```

#### 4. **AutoAugment / RandAugment**
```python
# 자동으로 최적 증강 정책 학습
from albumentations.pytorch import ToTensorV2
# Albumentations도 AutoAugment 지원
```

---

### 🔍 디버깅 팁

**증강이 너무 강해서 성능이 떨어진다면:**
1. 섹션 9의 시각화로 증강 결과 확인
2. `p` (확률) 파라미터 조정
3. 더 약한 레벨로 변경

**학습이 불안정하다면:**
1. Learning rate 감소
2. Batch size 증가
3. 증강 강도 감소

---

### 📚 참고 자료

- [Albumentations 공식 문서](https://albumentations.ai/docs/)
- [Albumentations 예제 모음](https://albumentations.ai/docs/examples/)
- [문서 이미지 증강 Best Practices](https://arxiv.org/abs/2106.08322)
